# **RAG model - NLP - házi feladat**
AX83OL

## Rövid ismertető

A következő colbertv2.0 ragatouille előtanított model a PTE valamint a PTE EHÖK hivatalos szabályzatain lett továbbtanítva.

Az alábbi weboldalak alatt lévő dokumentumokat használja:

*   A Pécsi Tudományegyetem Szervezeti és Működési Szabályzata és mellékletei:

    https://adminisztracio.pte.hu/adminisztracio/szabalyzatok_utasitasok/szervezeti_es_mukodesi_szabalyzat_es_mellekletek

*   PTE EHÖK Dokumentumtár:

    https://pteehok.hu/dokumentumtar/

Az összes dokumentum manuálisan le lett töltve a két oldalról, majd feltöltve ZIP fájlként a Google Drive-ra, onnan tud ez a notebook hozzáférni.

A továbbtanításhoz használni kívánt PDF-ek szerencsére szövegként tartalmaznak mindent, így azok felhasználhatóak voltak. (Illetve volt talán 2-3 melléklet ami nem, de azok amúgy sem túl kardinális témáról szólnak, így azokat kihagytam.)

Az összes használt doksi (=tanítóhalmaz) együttvéve kb. 1800 oldalt tesz ki.

# MEGVALÓSÍTÁS:

Feltelepítjük a két alapvetően szükséges modult (CUDA-hoz és RAG-hoz kell).

In [1]:
!pip install optimum auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.0/410.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 9.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.

In [2]:
!pip install ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.8 

Betöltjük az adott modelt.

In [3]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Kell majd a PyPDF2 modul a PDF-ek feldolgozásához, így azt is feltelepítjük.

In [4]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00


Készítünk egy függvényt ami a Google Drive-ról letöltött és már kicsomagolt mappában lévő összes PDF doksin végigmegy, majd azokat betölti egy string változóba.

In [5]:
import os
from PyPDF2 import PdfReader

def read_pdfs(directory):
    """
    Read all PDF files in the specified directory and concatenate their contents into a single string.

    :param directory: str - Path to the directory containing PDF files.
    :return: str - Concatenated text content of all PDF files.
    """
    full_text = ""

    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)

        # Check if the file is a PDF
        if filepath.endswith(".pdf"):
            with open(filepath, "rb") as file:
                reader = PdfReader(file)

                # Iterate through each page of the PDF
                for page_num in range(len(reader.pages)):
                    page = reader.pages[page_num]
                    full_text += page.extract_text()

    return full_text

Letöltjük a forrásadatokat. (Nálam már le van, tehát nekem felesleges.)

In [6]:
!gdown 16Pn9bYZSY3no-QXvwKlsK0ORrdaG0TQX

Downloading...
From (original): https://drive.google.com/uc?id=16Pn9bYZSY3no-QXvwKlsK0ORrdaG0TQX
From (redirected): https://drive.google.com/uc?id=16Pn9bYZSY3no-QXvwKlsK0ORrdaG0TQX&confirm=t&uuid=297331d2-9094-479a-b711-b872ba0fef8d
To: /content/Szabalyzatok.zip
100% 39.3M/39.3M [00:00<00:00, 80.5MB/s]


Kicsomagoljuk a letöltött ZIP-et, hogy alkalmazni tudjuk az így kapott mappára a read_pdfs függvényt.

In [7]:
import zipfile

# A ZIP fájl elérési útja
zip_file_path = "Szabalyzatok.zip"

# A célkönyvtár, ahova kicsomagoljuk a fájlokat
extract_to_directory = ""

# Megnyitjuk a ZIP fájlt és kicsomagoljuk a tartalmát
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)

print("A ZIP fájl sikeresen kicsomagolva.")

A ZIP fájl sikeresen kicsomagolva.


Alkalmazzuk az előbb említett függvényt.

In [8]:
# Directory containing the PDF files
pdf_directory = "Szabalyzatok"

# Read all PDF files in the directory and concatenate their contents
full_document = read_pdfs(pdf_directory)

# Check the length of the concatenated text
print(len(full_document))
type(full_document)

5397394


str

In [9]:
full_document

' \n \n \n \n \n \n \n \n \n \nA Pécsi Tudományegyetem  \n \nSzervezeti és Működési Szabályzatának 37. számú melléklete  \n \n \n \nA Klinikai Központ  \nSzervezeti és Működési Szabályzata  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nPécs 20 21. \n \n2024. április 8.  napjától  hatályos változat  \n \n \n \n \n  \nPreambulum1 2 3 \n \nA nemzeti felsőoktatásról szóló 2011. évi CCIV. törvény, az egészségügyről szóló 1997. évi CLIV. törvény  \n(Eütv.) , az egészségügyi szolgálati jogviszonyról szóló 2020. évi C. törvény (Eszjtv.)  rendelkezései, a \nközfeladatot ellátó közérdekű vagyonkezelő alapítványokról szóló 2021. évi IX. törvény, valamint a Pécsi \nTudományegyetem Szervezeti és Működési Szabályzatának  rendelkezései  alapján a Pécsi \nTudományegyetemen (továbbiakban: Egyetem) a z egész ségügyi szolgáltatást végző és a felsőoktatási \nképzési programban részt vevő kliniká k, kórházak , diagnosztikai egysége k és az egészségügyi szolgáltatás \nszervezéséhez kapcsolódó egyéb e

A tanítóhalmaz indexeléséhez eltávolítjuk a ragatoullie modullal automatikusan installált CPU modult, majd meggyőződünk, hogy a GPU-s verzió fent van-e.

In [10]:
!pip uninstall faiss-cpu
!pip install faiss-gpu

Found existing installation: faiss-cpu 1.8.0
Uninstalling faiss-cpu-1.8.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/faiss/*
    /usr/local/lib/python3.10/dist-packages/faiss_cpu-1.8.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/faiss_cpu.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.10/dist-packages/faiss_cpu.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.10/dist-packages/faiss_cpu.libs/libquadmath-96973f99.so.0.0.0
Proceed (Y/n)? Y
  Successfully uninstalled faiss-cpu-1.8.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 0:00:00


### Indexeljük a tanítóhalmazt, itt hossznak 180-at adtam meg: ez azt jelenti, hogy milyen hosszú részekre darabolja fel a tanítóhalmazt. => Majd a válaszok milyen hosszúak legyenek max.

In [11]:
RAG.index(collection=[full_document], index_name="PTE", max_document_length=180, split_documents=True)

---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Apr 09, 21:24:09] #> Creating directory .ragatouille/colbert/indexes/PTE 


[Apr 09, 21:24:11] [0] 		 #> Encoding 18195 passages..
[Apr 09, 21:25:20] [0] 		 avg_doclen_est = 121.39813232421875 	 len(local_sample) = 18,195
[Apr 09, 21:25:27] [0] 		 Creating 16,384 partitions.
[Apr 09, 21:25:27] [0] 		 *Estimated* 2,208,839 embeddings.
[Apr 09, 21:25:27] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/PTE/plan.json ..
used 20 iterations (3.8588s) to cluster 2158839 items into 16384 clusters
[Apr 09, 21:25:33] Loading decompr

0it [00:00, ?it/s]

[Apr 09, 21:29:14] [0] 		 #> Encoding 18195 passages..


1it [01:10, 70.40s/it]
100%|██████████| 1/1 [00:00<00:00, 176.12it/s]


[Apr 09, 21:30:25] #> Optimizing IVF to store map from centroids to list of pids..
[Apr 09, 21:30:25] #> Building the emb2pid mapping..
[Apr 09, 21:30:25] len(emb2pid) = 2208839


100%|██████████| 16384/16384 [00:00<00:00, 62080.52it/s]

[Apr 09, 21:30:26] #> Saved optimized IVF to .ragatouille/colbert/indexes/PTE/ivf.pid.pt


Done indexing!


'.ragatouille/colbert/indexes/PTE'

A következő függvénnyel tudunk kérdéseket feltenni a megtanult doksikkal kapcsolatban.

In [12]:
def inputFunction(question):
  k = 3 # How many documents you want to retrieve, defaults to 10, we set it to 3 here for readability
  results = RAG.search(query=question, k=k)
  return results[0]['content'].replace("\n","").replace("   ","  ").replace("  "," ")

Tesztként:

In [13]:
result = inputFunction("Mikor utasítja el érdemi vizsgálat nékül az EHÖK a kollégiumi jelentkezésemet?")
print(result)

Loading searcher for index PTE for the first time... This may take a few seconds
[Apr 09, 21:30:36] #> Loading codec...
[Apr 09, 21:30:36] #> Loading IVF...
[Apr 09, 21:30:36] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 91.24it/s]

[Apr 09, 21:30:36] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00,  5.03it/s]


Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Mikor utasítja el érdemi vizsgálat nékül az EHÖK a kollégiumi jelentkezésemet?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2771, 21815, 28981, 28032,  3900,  3449,  9413,  3207,
         4328, 26619, 28745,  7911,  2102, 11265,  5283,  2140, 17207, 15501,
         6559,  1037, 12849,  6216,  5856, 12717, 15333, 16136,  3489, 11254,
        21382,   102], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')

A hibásan kitöltött pályázati űrlapot a Küldöttgyűlés érdemi vizsgálat nélkül elutasítja. A hallgató a PTE EHÖK Küldöttgyűlésének döntése ellen, jogszabály vagy szabályzatsértésre hivatkozássa l fellebbezéssel élhet, melyet a Másodfokú Tanulmányi Bizottságnak címezve a PTE EHÖK Küldöttgyűléséhez kell benyújtani a tudomásszerz

## Betöltünk a Hugging Face-ről egy transformers szöveggeneráló modelt amely támogat magyar nyelvet (és remélhetőleg elbírja a Colab rendszere is). Majd átadjuk ennek az előző colbertv2.0 RAG model válaszát.

In [28]:
from transformers import pipeline

def theFunction(Question):

  answerSkeleton = inputFunction(Question)

  prompt = f"{Question} válaszold meg a következő szöveg alapján, csak a szöveget használhatod fel: {answerSkeleton}"
  generator = pipeline(task="text-generation", model="NYTK/PULI-GPT-2", max_length=103)

  return generator(prompt)[0]["generated_text"]

In [29]:
theFunction("Mikor utasítja el érdemi vizsgálat nékül az EHÖK a kollégiumi jelentkezésemet?")

Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.


'Mikor utasítja el érdemi vizsgálat nékül az EHÖK a kollégiumi jelentkezésemet? válaszold meg a következő szöveg alapján, csak a szöveget használhatod fel: A hibásan kitöltött pályázati űrlapot a Küldöttgyűlés érdemi vizsgálat nélkül elutasítja. A hallgató a PTE EHÖK Küldöttgyűlésének döntése ellen, jogszabály vagy szabályzatsértésre hivatkozássa l fellebbezéssel élhet, melyet a Másodfokú Tanulmányi Bizottságnak címezve a PTE EHÖK Küldöttgyűléséhez kell benyújtani a tudomásszerzéstől számított 15 napon belül. A'

A Gradio-t is fel kell telepítenünk a webes GUI miatt.

In [30]:
!pip install gradio

Megvalósítunk egy egyszerű webes GUI-t:

In [31]:
import gradio as gr

# Létrehozunk egy Gradio interfészt a függvényünkkel
interface = gr.Interface(
    fn=theFunction,
    inputs="text",
    outputs="text",
    title="RAG Model Summary Generator (PTE admin edition)",
    description="Generates a summary for a given input text using the RAG model. This RAG model trained on many administrative documents of PTE and PTE EHÖK.",
)

# A Gradio interfész indítása
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://65cf8cdc93e5dc4829.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
